In [ ]:
from CADETProcess.processModel import Inlet, Outlet, ComponentSystem, LumpedRateModelWithoutPores, StericMassAction

component_system = ComponentSystem(["Salt", "ProteinA", "ProteinB", "ProteinC"])

inlet = Inlet(component_system, "inlet")
inlet.flow_rate = 1 / 60 / 1000 / 1000

outlet = Outlet(component_system, "outlet")

column = LumpedRateModelWithoutPores(component_system, "column")

column.total_porosity = 0.4
column.axial_dispersion = 1e-6
column.length = 0.014
column.diameter = 0.01
column.c = [50, 0, 0, 0]

binding = StericMassAction(component_system, "binding")
binding.is_kinetic = True
binding.adsorption_rate = [0, 1e-5, 1e-1, 1e-3]
binding.desorption_rate = [0, 1, 1, 1]
binding.characteristic_charge = [0, 5, 1, 7]
binding.steric_factor = [0, 1, 1, 1]
binding.capacity = 1200

column.binding_model = binding

column.q = [50, 0, 0, 0]

from CADETProcess.processModel import FlowSheet, Process

flow_sheet = FlowSheet(component_system, "flow_sheet")

flow_sheet.add_unit(inlet)
flow_sheet.add_unit(column)
flow_sheet.add_unit(outlet)

flow_sheet.add_connection(inlet, column)
flow_sheet.add_connection(column, outlet)

process = Process(flow_sheet, "process")

gradient_column_volumes = 15
gradient_duration = gradient_column_volumes * column.volume / inlet.flow_rate[0]

process.cycle_time = 90 + gradient_duration + 90

# Dynamic Events - Exercises

## Exercise 1: Load - Wash - Elute

Extend the SMA model from the previous exercise and divide it into sections:
- __a load section__: during which the column is loaded with 1 mol of Protein while being held at 50 mM salt concentration
- __a wash section__: during which the column is washed with 50 mM salt buffer for 3 column volumes to allow non-binding protein to elute
- __an elute section__: during which the column is brought to 1000 mM salt buffer to elute the bound proteins.

In [ ]:
# add load
total_protein = 0.001  # mol
load_duration = 10  # sec
protein_concentration = 1.0

process.add_event(
    name="load",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [50, 0, 0, 0],
        [protein_concentration, 0, 0, 0],
        [protein_concentration, 0, 0, 0],
        [protein_concentration, 0, 0, 0]
    ],
    time=0
)
# add wash
process.add_event(
    name="wash",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [50, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    time=10
)

start_concentration = 50
end_concentration = 1000
gradient_duration

slope = (end_concentration - start_concentration) / gradient_duration

# add gradient
process.add_event(
    name="gradient_start",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [50, slope, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    time=90
)

# stop gradient
process.add_event(
    name="gradient_end",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [1000, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    time=90 + gradient_duration
)

from CADETProcess.simulator import Cadet
simulator = Cadet()
simulation_results = simulator.simulate(process)
simulation_results.solution.outlet.outlet.plot()

In [ ]:
from CADETProcess.plotting import SecondaryAxis

sec = SecondaryAxis()
sec.components = ['Salt']
sec.y_label = '$c_{salt}$'

_ = simulation_results.solution.outlet.outlet.plot(secondary_axis=sec, x_axis_in_minutes=False)